In [2]:
from transformers import GPT2ForSequenceClassification, Trainer, GPT2Config, GPT2Tokenizer, TrainingArguments

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [3]:
import pandas as pd
import numpy as np

In [4]:
import torch
from torch.utils.data import Dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [5]:
config = GPT2Config.from_json_file('/kaggle/input/gpt2-trained-weights/config.json')
model = GPT2ForSequenceClassification(config)
model.load_state_dict(torch.load('/kaggle/input/gpt2-trained-weights/pytorch_model.bin'))

<All keys matched successfully>

In [6]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [7]:
model.eval()

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50258, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=5, bias=False)
)

In [8]:
X_train = pd.read_csv('/kaggle/input/splitted-dataset/Dataset/Dataset/X_train.csv')
y_train = pd.read_csv('/kaggle/input/splitted-dataset/Dataset/Dataset/y_train.csv')
X_test = pd.read_csv('/kaggle/input/splitted-dataset/Dataset/Dataset/X_test.csv')
y_test =pd.read_csv('/kaggle/input/splitted-dataset/Dataset/Dataset/y_test.csv')
X_train_discarded = pd.read_csv('/kaggle/input/splitted-dataset/Dataset/Dataset/X_train_discarded.csv')
y_train_discarded = pd.read_csv('/kaggle/input/splitted-dataset/Dataset/Dataset/y_train_discarded.csv')

In [9]:
X_train = X_train.drop('Unnamed: 0', axis=1)
y_train = y_train.drop('Unnamed: 0', axis=1)
X_test = X_test.drop('Unnamed: 0', axis=1)
y_test = y_test.drop('Unnamed: 0', axis=1)
X_train_discarded = X_train_discarded.drop('Unnamed: 0', axis=1)
y_train_discarded = y_train_discarded.drop('Unnamed: 0', axis=1)

In [11]:
# GPT-2 tokenizer does not have a padding token by default. We need to add one.
# We add a pad token and also update the model config.
tokenizer.add_special_tokens({'pad_token': '<PAD>'})
tokenizer.model_max_length = 512

# Tokenize the text data
test_encodings = tokenizer(X_test['Text'].tolist(), truncation=True, padding=True, max_length=512)

# Convert labels to integer values
le = LabelEncoder()
test_labels = le.fit_transform(y_test['PICO_Entity'])

# Prepare the dataset
class MyDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

test_dataset = MyDataset(test_encodings, test_labels)

In [12]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import matthews_corrcoef, cohen_kappa_score
from scipy.stats import pointbiserialr, pearsonr, spearmanr, kendalltau
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')
    f1 = f1_score(labels, predictions, average='weighted')
    accuracy = accuracy_score(labels, predictions)
    kappa = cohen_kappa_score(labels, predictions)
    conf_matrix = confusion_matrix(labels, predictions)

    mcc = []
    pearson = []
    spearman = []
    kendall = []
    point_biserial = []
    for class_index in range(logits.shape[1]): # iterate over classes
        binary_labels = (labels == class_index)
        binary_predictions = (predictions == class_index)

        mcc.append(matthews_corrcoef(binary_labels, binary_predictions))
        pearson.append(pearsonr(binary_labels, binary_predictions)[0])
        spearman.append(spearmanr(binary_labels, binary_predictions)[0])
        kendall.append(kendalltau(binary_labels, binary_predictions)[0])
        point_biserial.append(pointbiserialr(binary_labels, binary_predictions)[0])

    return {
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'accuracy': accuracy,
        'kappa': kappa,
        'confusion_matrix': conf_matrix,
        'mcc': np.mean(mcc),  # Take the mean of the per-class MCCs
        'pearson': np.mean(pearson),  # Take the mean of the per-class Pearson coefficients
        'spearman': np.mean(spearman),  # Take the mean of the per-class Spearman coefficients
        'kendall': np.mean(kendall),  # Take the mean of the per-class Kendall coefficients
        'point_biserial': np.mean(point_biserial),  # Take the mean of the per-class Point Biserial coefficients
    }

In [13]:
training_args = TrainingArguments(
    output_dir='./results',  # Output directory
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_steps=3000,  # Save every 1000 steps
)
# Create the trainer and fine-tune the model
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics
)

In [14]:
# Evaluate the model
eval_results = trainer.evaluate(test_dataset)

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Trainer is attempting to log a value of "[[ 44   3   0   0  22]
 [  0  54   0   0  15]
 [  0   0  16   1   8]
 [  0   0   0   2   3]
 [ 22  11  11   0 788]]" of type <class 'numpy.ndarray'> for key "eval/confusion_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [15]:
eval_results = trainer.evaluate(test_dataset)
print(eval_results)

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Trainer is attempting to log a value of "[[ 44   3   0   0  22]
 [  0  54   0   0  15]
 [  0   0  16   1   8]
 [  0   0   0   2   3]
 [ 22  11  11   0 788]]" of type <class 'numpy.ndarray'> for key "eval/confusion_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.2467719167470932, 'eval_precision': 0.9031719308669772, 'eval_recall': 0.904, 'eval_f1': 0.9033668654969342, 'eval_accuracy': 0.904, 'eval_kappa': 0.6740370511218559, 'eval_confusion_matrix': array([[ 44,   3,   0,   0,  22],
       [  0,  54,   0,   0,  15],
       [  0,   0,  16,   1,   8],
       [  0,   0,   0,   2,   3],
       [ 22,  11,  11,   0, 788]]), 'eval_mcc': 0.6375251137269544, 'eval_pearson': 0.6375251137269542, 'eval_spearman': 0.6375251137269544, 'eval_kendall': 0.6375251137269544, 'eval_point_biserial': 0.6375251137269542, 'eval_runtime': 4.8752, 'eval_samples_per_second': 205.122, 'eval_steps_per_second': 1.641}
